<a href="https://colab.research.google.com/github/Iispar/review-summary-API/blob/main/BERT-finetuned-model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!pip3 install -q transformers datasets evaluate
!pip install optuna
import datasets
import numpy as np
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Preprocessing

The dataset includes reviews from multiple languages so we only import the english ones. The dataset also includes alot of useless data for us, we only need the reviews and their ratings so lets process everything else out.

In [4]:
dataset = datasets.load_dataset('amazon_reviews_multi', name='en'); # imports the dataset.
# check it works
print(dataset);

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/200000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Dataset amazon_reviews_multi downloaded and prepared to /root/.cache/huggingface/datasets/amazon_reviews_multi/en/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 200000
    })
    validation: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 5000
    })
})


In [5]:
dataset = dataset.shuffle() # shuffle the dataset for safety.
dataset = dataset.remove_columns(['review_id', 'product_id', 'reviewer_id', 'language', 'product_category']) # removes everything that we don't need
dataset = dataset.rename_column('stars', 'label') # rename stars to label so it is a bit more understandable
# an error was coming up because of the labels were 1-5 and not 0-4 so let's change that for all.
# at the same time lets add the title to the start of the review with an :.

def addTitle_and_changeLables(example):
  example['label'] = example['label'] - 1; # lower the label by one so we get 0-4
  example['review_body'] = f'{example['review_title']}: {example['review_body']}'; # add title to review body
  return example # return the item
dataset = dataset.map(addTitle_and_changeLables) # map the function to all.
dataset = dataset.remove_columns(['review_title']) # now we can also remove the title

# let's check that it worked.
print(dataset)
print(dataset['train'][3]) 

Map:   0%|          | 0/200000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'review_body'],
        num_rows: 200000
    })
    validation: Dataset({
        features: ['label', 'review_body'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['label', 'review_body'],
        num_rows: 5000
    })
})
{'label': 1, 'review_body': 'It’s ok...: Smells ok. Makes skin soft. I think that’s about it. I didn’t notice any help in the cellulite department.'}


# Tokenization and padding

In [6]:
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased') # get the basic AutoTokenizer 
data_collator = DataCollatorWithPadding(tokenizer=tokenizer) # get the data collator for the padding and set the tokenizer as ours.

In [10]:
def preprocess_function(examples):
    return tokenizer(examples['review_body'], truncation=True) # tokenizes one example

In [ ]:
dset_tokenized = dataset.map(preprocess_function, batched=True) # tokenize the whole dataset with map
print(dset_tokenized['train'][0]) # lets check that it worked

In [13]:
model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=5) # load the bert model with weights

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'classifier

# Fine tuning the BERT model for our classification

In [19]:
# evaluation
accuracy = evaluate.load('accuracy');
def compute_accuracy(outputs_and_labels):
    outputs, labels = outputs_and_labels;
    predictions = np.argmax(outputs, axis=-1); #pick the index of the "winning" label
    return accuracy.compute(predictions=predictions, references=labels); # calc accuracy

# Training params. We optimize these later
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy = 'steps',
    logging_strategy = 'steps',
    eval_steps = 500,
    logging_steps = 500,
    learning_rate=1e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    max_steps = 20000,
    num_train_epochs=5,
    weight_decay=0.01,
  )

# Set the trainer
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = dset_tokenized['train'],
    eval_dataset = dset_tokenized['test'],
    tokenizer = tokenizer,
    data_collator = data_collator,
    compute_metrics = compute_accuracy,
)

# train the model
trainer.train()

Step,Training Loss,Validation Loss,Accuracy
500,0.684600,1.059772,0.583600
1000,0.952100,0.936430,0.607800
1500,0.906700,0.939449,0.598400
2000,0.923800,0.889306,0.608800
2500,0.899000,0.899476,0.617000
3000,0.891600,0.898800,0.620200
3500,0.901300,0.885515,0.609800
4000,0.871500,0.890550,0.608800
4500,0.869300,0.862399,0.633600
5000,0.882800,0.875236,0.620400


TrainOutput(global_step=20000, training_loss=0.8053289947509765, metrics={'train_runtime': 5529.9638, 'train_samples_per_second': 57.867, 'train_steps_per_second': 3.617, 'total_flos': 1.255790231830848e+16, 'train_loss': 0.8053289947509765, 'epoch': 1.6})